In [1]:
import requests
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import re
from itertools import cycle
import traceback
import smtplib

In [2]:
# url = 'https://ottawapoliceboard.ca/opsb-cspo/meetings.html'
# url = 'https://pub-ottawa.escribemeetings.com/?Expanded=Ottawa%20Police%20Services%20Board&Year=2023'
# url = 'https://pub-ottawa.escribemeetings.com/Meeting.aspx?Id=3760766e-baa1-4574-a3c3-da5e7879c8c6&Agenda=Agenda&lang=English'

current_year = 2022
reg_meet = []
meeting_no = 1

while current_year > 2010:
  url = f'https://ottawapoliceboard.ca/opsb-cspo/{current_year}-meetings.html'

  options = webdriver.ChromeOptions()
  options.add_argument("--disable-infobars")
  options.add_argument("--start-maximized")
  options.add_argument("--disable-extensions")
  options.add_argument('--window-size=1920,1080')
  options.add_argument("--headless")
  driver = webdriver.Chrome(options = options)

  driver.get(url)
  page_src = driver.page_source
  driver.close()
  soup = BeautifulSoup(page_src, 'html.parser')

  # print('$$$$$$$$$$$$$$$', soup.find_all('div', text=re.compile('Regular Meeting')))
  # print('$$$$$$$$$$$$$$$', soup.find_all('a', attrs = {''} text=re.compile('Regular Meeting')))
  # print('$$$$$$$$$$$$$$$', soup.find_all("div", class_="meeting-date"))
  # print('$$$$$$$$$$$$$$$', soup.find_all(["a", "div"], {"class": ["Ottawa Police Services Board", "meeting-date"]}))

  meetings_list = soup.find('table').find('tbody').find_all('tr')

  for index, tr in enumerate(reversed(meetings_list)):
      td = tr.find_all('td')
      if td[3].get_text().strip() == 'Regular Meeting':
        if current_year < 2013: date = f'{td[0].get_text().strip()}, {current_year}'
        else: date = td[0].get_text().strip()
        reg_meet.append({
          'Meeting #': meeting_no,
          'Date': date,
          'Location': td[1].get_text().strip(),
          'Time': td[2].get_text().strip(),
          'Meeting Type': td[3].get_text().strip(),
          'Meeting Page': td[0].find('a').get('href')
          })
        meeting_no = meeting_no + 1
  print(f'Regular Meetings of Year {current_year} processed.')
  current_year = current_year - 1

df = pd.DataFrame(reg_meet)
df.to_excel('regular_meetings.xlsx')

Regular Meetings of Year 2022 processed.
Regular Meetings of Year 2021 processed.
Regular Meetings of Year 2020 processed.
Regular Meetings of Year 2019 processed.
Regular Meetings of Year 2018 processed.
Regular Meetings of Year 2017 processed.
Regular Meetings of Year 2016 processed.
Regular Meetings of Year 2015 processed.
Regular Meetings of Year 2014 processed.
Regular Meetings of Year 2013 processed.
Regular Meetings of Year 2012 processed.
Regular Meetings of Year 2011 processed.


In [134]:
def send_mail(): 
    to = 'savan.kotadiya2000@gmail.com'
    gmail_user = 'dedpoolop@gmail.com'
    gmail_pwd = 'dmittwestwlfmojs'
    smtpserver = smtplib.SMTP("smtp.gmail.com", 587)
    smtpserver.ehlo()
    smtpserver.starttls()
    smtpserver.ehlo()
    smtpserver.login(gmail_user, gmail_pwd)
    header = 'To:' + to + '\n' + 'From: ' + gmail_user + '\n' + 'Subject:testing \n'
    msg = header + '\n this is test msg from Capstone Project \n\n'
    smtpserver.sendmail(gmail_user, to, msg)
    filename = "meeting_output.xlsx"
    with open('meeting_output.xlsx', 'rb') as f:
        file_data = f.read()
    msg.add_attachment(file_data, maintype="application", subtype="xlsx", filename='meeting_output.xlsx')
    smtpserver.quit()
    print ('done!')